In [ ]:
import pandas as pd
import datetime
import requests
import json
import math
import numpy as np

## key 저장

In [ ]:
f = open("keyfile.txt", 'r')
key = {}
for i in range(1, 11):
    key[i] = f.readline().rstrip()
    print('key'+str(i))
f.close()

In [ ]:
key

In [ ]:
key[1]

## 데이터 목록
### 1. 영화 목록
* INPUT
    - 키 값, 현재 페이지, 결과 ROW 의 개수, 영화명, 감독명, 조회시작 개봉연도(YYYY), 조회종료 개봉연도(YYYY), 조회시작 제작연도(YYYY), 조회종료 제작연도(YYYY), 국적코드, 영화유형코드

* OUTPUT
    - 영화코드, 영화명(국문),영화명(영문), 제작연도, 개봉연도, 영화유형, 제작상태, 제작국가(전체), 영화장르(전체), 대표 제작국가명, 대표 장르명, 영화감독, 영화감독명, 제작사, 제작사 코드, 제작사명

### 2. 영화 상세정보
* INPUT
    - 키 값, 영화코드
    
* OUTPUT
    - 영화코드, 영화명(국문), 영화명(영문), 영화명(원문), 제작연도, 상영시간, 개봉연도, 제작상태명, 영화유형명, 제작국가, 제작국가명, 장르명, 감독, 감독명, 감독명(영문), 배우, 배우명, 배우명(영문), 배역명, 배역명(영문), 상영형태 구분1, 상영형태 구분2, 상영형태명, 심의정보, 심의번호, 관람등급 명칭, 참여 영화사, 참여 영화사 코드, 참여 영화사명, 참여 영화사명(영문), 참여 영화사 분야명, 스텝, 스텝명, 스텝명(영문), 스텝역할명
    
### 3. 영화사 목록
* INPUT
    - 키 값, 현재 페이지(default : “1”), 결과 ROW 의 개수(default : “10”), 영화사명, 대표자명, 분류코드(default: 전체)
* OUTPUT
    - 영화사 코드, 영화사명, 영화사명(영문), 영화사 분류, 대표자명, 필모리스트

### 4. 영화사 상세정보
* INPUT
    - 키 값, 영화사코드
* OUTPUT
    - 영화사 코드, 영화사명, 영화사명(영문), 대표자명, 영화사 분류, 영화사 분류명, 영화사 필모, 참여 영화코드, 참여 영화명, 영화사 참여 분류명

### 5. 영화인 목록
* INPUT
    - 키 값, 현재 페이지를 지정(default : “1”), 결과 ROW 의 개수(default : “10”), 영화인명, 필모리스트
* OUTPUT
    - 영화인 코드, 영화인명, 영화인명(영문), 분야, 필모리스트

### 6. 영화인 상세정보
* INPUT
    - 키 값, 영화인코드
* OUTPUT
    - 영화인 코드, 영화인명, 영화인명(영문), 성별, 영화인 분류명, 영화인 필모, 참여 영화코드, 참여 영화명, 참여 분야, 관련 URL


### # 주의 # 한 테스크가 끝날 때마다 csv로 저장하고 다음 작업 시작할 때는 저장된 csv 불러와서 시작해야 합니다.

In [ ]:
directors_df = pd.read_csv('directors_2019_ko.csv')

In [ ]:
actors_df = pd.read_csv('actors_2019_ko.csv')

In [ ]:
staffs_df = pd.read_csv('staffs_2019_ko.csv')

In [ ]:
companys_df = pd.read_csv('companys_2019_ko.csv')

<br>\
<br>\
<br>
## 영화인 정보 가져오기

위에서 영화 상세정보 데이터에서 감독, 배우, 스태프 각각의 정보를 가져왔죠. 그 정보를 이용해서 영화인의 코드를 알아낼 겁니다. 번거롭지만 영화인 코드는 영화인 세부정보를 가져오기 위해 꼭 필요한 데이터에요.

In [ ]:
def get_staffs(movieNm, peopleNm):
    url = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/people/searchPeopleList.json'

    # 영화인의 기본정보를 가져오기 위해서는 정보를 알고 싶은 사람의 '이름'과 '참여했던 작품 이름'이 있어야 합니다.
    # 영화 이름만으로 검색하면 영화에 참여했던 모든 스태프를 가져올 수 있지만 동명의 영화가 함께 검색되어 특정 영화의 정보를 가져오기에는 무리가 있습니다.
    # 때문에 특정하게 알고싶은 사람이 이름과 작품의 이름으로 특정 사람을 한정지어 가져오는 것이 낫다고 판단하여 짜여진 함수입니다.
    payload = {
        'key': key[1]
        , 'curPage': 1 # 한사람만 검색되므로 1페이지를 넘어가는 경우가 없을 거라고 가정하고 진행
        , 'itemPerPage': '50' # 한사람만 검색되므로 데이터 갯수가 50개를 넘어가는 경우가 없을 거라고 가정하고 진행
        , 'peopleNm': peopleNm
        , 'filmoNames': movieNm
    }

    res = requests.get(url, params=payload).json()
    
    return res['peopleListResult']

In [ ]:
def get_staffs_code(data, listNm):
    num = 1
    
    # 이름과 작품으로 영화인 정보에 접근할 것을 알고 있었기 때문에 위에서 감독, 배우, 스태프들의 정보를 담을 때 각각의 사람들이 참여했던 작품의 이름도 함께 저장이 되어있습니다.
    # 때문에 directors_df, actors_df, staffs_df 어느것을 넣더라도 똑같이 작동할 수 있어요.
    
    # get_staffs_code()에 directors_df가 입력되면
    # 아래 반복문에서 이름 컬럼과 작품 컬럼을 각각 하나씩 튜플로 묶어 반복문을 실행합니다.

    for movieNm, peopleNm in zip(data['movieNm'], data['peopleNm']):
        person_info = {}
        
        # 한번 for문이 돌때마다 코드를 검색할 사람의 이름과 작품이 뽑히고, 해당 정보를 get_staffs() 함수에 집어넣으면 해당 영화인의 기본정보를 얻을 수 있습니다.
        staff = get_staffs(movieNm, peopleNm)['peopleList'][0]

        person_info['peopleCd'] = staff['peopleCd']
        person_info['movieNm'] = movieNm
        person_info['peopleNm'] = peopleNm
        person_info['repRoleNm'] = staff['repRoleNm']
        person_info['filmoNames'] = staff['filmoNames']

        listNm.append(person_info)
        
        print(peopleNm, num)
        num += 1

    return ''

In [ ]:
directors_df

In [ ]:
actors_df

In [ ]:
staffs_df

In [ ]:
directors_df[:3]

In [ ]:
#test
# res = get_staffs_code(directors_df[:3])
# res

### TODO. get_staffs_code() 함수를 이용해 2019 directors, actors, staffs의 영화인 코드를 가져오기.

* 각각의 데이터 프레임에서 데이터의 갯수를 보고 데이터 프레임 전체를 한 번에 돌릴지 나누어서 몇 번에 돌릴 수 있을지 판단해서 사용하는 key 조정을 해야 합니다.

* get_staffs_code() 함수의 결과로는 리스트가 반환됩니다.

ex) 한번에 돌릴 수 있을 때:
```python
result = get_staffs_code(directors_df)
```

ex) 한번에 돌릴 수 없을 때:
```python
result1 = get_staffs_code(directors_df.iloc[:3000]) # def get_staffs_code가 정의 되어있는 코드에서 key 값을 새 것으로 바꿈.
result2 = get_staffs_code(directors_df.iloc[3000:6000])

result = result1 + result2
```

Q1) 2019 감독들의 영화인 코드 가져오기

In [ ]:
directorCd_list = []
get_staffs_code(directors_df, directorCd_list)

Q2) 2019 배우들의 영화인 코드 가져오기

In [ ]:
actorCd_list = []
get_staffs_code(actors_df, actorCd_list)

Q3) 2019 스태프들의 영화인 코드 가져오기

In [ ]:
staffCd_list1 = []
get_staffs_code(staffs_df[:3000], staffCd_list1)

In [ ]:
# staffCd_list2 = []
# get_staffs_code(staffs_df[3000:6000], staffCd_list2)

In [ ]:
# staffCd_list3 = [] 
# get_staffs_code(staffs_df[6000:], staffCd_list3)

Q4) 2019 감독, 배우, 스태프들의 영화인 코드를 하나의 데이터 프레임으로 합쳐 csv로 저장해주세요. index=False로 저장해주세요.

해당 데이터프레임을 이용해 영화인 상세 데이터를 가져와야 합니다.

각각 csv로 저장하기

In [ ]:
directorCd_df = pd.DataFrame(directorCd_list)
directorCd_df.to_csv('directorCd_2019_list.csv', index=False)

In [ ]:
actor_df = pd.DataFrame(actorCd_list)
actor_df.to_csv('actorCd_2019_list.csv', index=False)

In [ ]:
staffCd1_df = pd.DataFrame(staffCd_list1)
staffCd1_df.to_csv('staffCd_2019_list1.csv', index=False)

In [ ]:
# staffCd2_df = pd.DataFrame(staffCd_list2)
# staffCd2_df.to_csv('staffCd_2019_list2.csv', index=False)

In [ ]:
# staffCd3_df = pd.DataFrame(staffCd_list3)
# staffCd3_df.to_csv('staffCd_2019_list3.csv', index=False)

### # 주의 # 한 테스크가 끝날 때마다 csv로 저장하고 다음 작업 시작할 때는 저장된 csv 불러와서 시작해야 합니다.

In [ ]:
directorCd_df = pd.read_csv('directorCd_2019_list.csv')

In [ ]:
actorCd_df = pd.read_csv('actorCd_2019_list.csv')

In [ ]:
staffCd1_df = pd.read_csv('staffCd_2019_list1.csv')

In [ ]:
# staffCd2_df = pd.read_csv('staffCd_2019_list2.csv')

In [ ]:
# staffCd3_df = pd.read_csv('staffCd_2019_list3.csv')

<br>\
<br>\
<br>
### 영화인 상세 가져오기

대망의 영화인 상세 데이터입니다.

In [ ]:
def get_staff_details(peopleCd):
    url = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/people/searchPeopleInfo.json'

    # 영화인 코드를 이용해 세부 데이터를 불러옵니다.
    payload = {
        'key': key[1]
        , 'peopleCd': peopleCd
    }

    res = requests.get(url, params=payload).json()
    
    return res['peopleInfoResult']['peopleInfo']

TODO) 아래 테스트는 임시 데이터로 한 테스트 입니다. 위에서 Q4번이 해결 됐으면 해당 csv로 치환해주세요.

In [ ]:
# 데이터 확인
directorCd_df

In [ ]:
# 임시로 테스트 데이터를 하나 불러와 저장했습니다.
# 어떤 데이터들이 어떤 형식으로 뽑혀나오는지 확인하고 상세정보를 어떻게 저장할지 설계합니다.
# 성별만 가져올까 하다가 여배우 남배우의 필모리스트도 비교해보면 좋을 듯 하여 저장하겠습니다.
get_staff_detail_test = get_staff_details(20167301)
get_staff_detail_test

In [ ]:
# 영화인 코드
get_staff_detail_test['peopleCd']

In [ ]:
# 성별
get_staff_detail_test['sex']

In [ ]:
# 필모그래피 리스트
for movie in get_staff_detail_test['filmos']:
    print(movie['movieCd'])
    print(movie['movieNm'])
    print(movie['moviePartNm'])

In [ ]:
# 2019년만 해도 감독, 배우, 스태프의 총 합은 3000을 훨씬 웃도는 숫자 입니다.
# 횟수 제한 문제로 3000번을 기준으로 잘라서 정보들을 저장한 후 나중에 합쳐주도록 하겠습니다.
# key 사용횟수가 3000회 이하로 남아있다면 가능 횟수에 맞춰서 코드 조정해주세요.
# 중간에 key 사용 가능 횟수가 모두 끝나면 저장하고 다른날 다시 불러와서 이어서 작업 해주세요.

director_detail_list = []
num = 1

for staffCd in directorCd_df['peopleCd'].iloc[:3000]:
    get_staff_detail_test = get_staff_details(staffCd)
    
    # 성별 리스트와 필모리스트를 따로 만들까 하다가 한 사람의 세부 정보인데 굳이 그럴 필요가 있나 싶어서 한꺼번에 저장합니다.
    # 필모리스트에 있는 영화들을 기준으로 해당 영화인이 참여 했던 작품 하나당 데이터 하나가 생성됩니다.
    # 데이터 한 행에는 '영화인코드, 이름, 성별, 참여영화코드, 참여영화이름, 참여했던역할(감독, 배우, 스태프 등)'이 들어갑니다.
    for movie in get_staff_detail_test['filmos']:
        person_info = {}

        person_info['peopleCd'] = get_staff_detail_test['peopleCd']
        person_info['peopleNm'] = get_staff_detail_test['peopleNm']
        person_info['sex'] = get_staff_detail_test['sex']

        person_info['movieCd'] = movie['movieCd']
        person_info['movieNm'] = movie['movieNm']
        person_info['moviePartNm'] = movie['moviePartNm']

        director_detail_list.append(person_info)
        
    print(get_staff_detail_test['peopleNm'], get_staff_detail_test['sex'])
    print(num)
    num += 1

In [ ]:
# 구간 바꿔서 다시 반복.
# get_staff_details 함수가 정의된 곳으로 돌아가 아직 사용 가능한 key 값으로 변경해줍니다.

actor_detail_list = []
num = 1

for staffCd in actorCd_df['peopleCd'].iloc[:3000]:
    get_staff_detail_test = get_staff_details(staffCd)
    
    for movie in get_staff_detail_test['filmos']:
        person_info = {}
        
        print(get_staff_detail_test['peopleNm'], get_staff_detail_test['sex'])

        person_info['peopleCd'] = get_staff_detail_test['peopleCd']
        person_info['peopleNm'] = get_staff_detail_test['peopleNm']
        person_info['sex'] = get_staff_detail_test['sex']

        person_info['movieCd'] = movie['movieCd']
        person_info['movieNm'] = movie['movieNm']
        person_info['moviePartNm'] = movie['moviePartNm']
        
        actor_detail_list.append(person_info)
                
    print(get_staff_detail_test['peopleNm'], get_staff_detail_test['sex'])
    print(num)
    num += 1

In [ ]:
# 구간 바꿔서 다시 반복.
# get_staff_details 함수가 정의된 곳으로 돌아가 아직 사용 가능한 key 값으로 변경해줍니다.

staff_detail_list_1 = []
num = 1

for staffCd in staffCd1_df['peopleCd'].iloc[:3000]:
    get_staff_detail_test = get_staff_details(staffCd)
    
    for movie in get_staff_detail_test['filmos']:
        person_info = {}
        
        print(get_staff_detail_test['peopleNm'], get_staff_detail_test['sex'])

        person_info['peopleCd'] = get_staff_detail_test['peopleCd']
        person_info['peopleNm'] = get_staff_detail_test['peopleNm']
        person_info['sex'] = get_staff_detail_test['sex']

        person_info['movieCd'] = movie['movieCd']
        person_info['movieNm'] = movie['movieNm']
        person_info['moviePartNm'] = movie['moviePartNm']
        
        staff_detail_list_1.append(person_info)
                
    print(get_staff_detail_test['peopleNm'], get_staff_detail_test['sex'])
    print(num)
    num += 1

In [ ]:
# 구간 바꿔서 다시 반복.
# get_staff_details 함수가 정의된 곳으로 돌아가 아직 사용 가능한 key 값으로 변경해줍니다.

# staff_detail_list_2 = []
# num = 1

# for staffCd in staffCd2_df['peopleCd'].iloc[:3000]:
#     get_staff_detail_test = get_staff_details(staffCd)
    
#     for movie in get_staff_detail_test['filmos']:
#         person_info = {}
        
#         print(get_staff_detail_test['peopleNm'], get_staff_detail_test['sex'])

#         person_info['peopleCd'] = get_staff_detail_test['peopleCd']
#         person_info['peopleNm'] = get_staff_detail_test['peopleNm']
#         person_info['sex'] = get_staff_detail_test['sex']

#         person_info['movieCd'] = movie['movieCd']
#         person_info['movieNm'] = movie['movieNm']
#         person_info['moviePartNm'] = movie['moviePartNm']
        
#         staff_detail_list_2.append(person_info)
                
#     print(get_staff_detail_test['peopleNm'], get_staff_detail_test['sex'])
#     print(num)
#     num += 1

In [ ]:
# 구간 바꿔서 다시 반복.
# get_staff_details 함수가 정의된 곳으로 돌아가 아직 사용 가능한 key 값으로 변경해줍니다.

# staff_detail_list_3 = []
# num = 1

# for staffCd in staffCd3_df['peopleCd'].iloc[:3000]:
#     get_staff_detail_test = get_staff_details(staffCd)
    
#     for movie in get_staff_detail_test['filmos']:
#         person_info = {}
        
#         print(get_staff_detail_test['peopleNm'], get_staff_detail_test['sex'])

#         person_info['peopleCd'] = get_staff_detail_test['peopleCd']
#         person_info['peopleNm'] = get_staff_detail_test['peopleNm']
#         person_info['sex'] = get_staff_detail_test['sex']

#         person_info['movieCd'] = movie['movieCd']
#         person_info['movieNm'] = movie['movieNm']
#         person_info['moviePartNm'] = movie['moviePartNm']
        
#         staff_detail_list_3.append(person_info)
                
#     print(get_staff_detail_test['peopleNm'], get_staff_detail_test['sex'])
#     print(num)
#     num += 1

<br>

### TODO. 영화인 상세정보를 뽑아주세요.

Q1) 2019 감독, 배우, 스태프 모두의 상세정보를 뽑아주세요.
각각의 리스트를 합칠 때는 아래 코드를 참고하세요.

```python
staff_detail_list_all = pd.DataFrame(staff_detail_list_1 + staff_detail_list_2 + staff_detail_list_3 + staff_detail_list_4 + staff_detail_list_5)
staff_detail_list_all
```

Q2) 가져온 데이터는 csv 형식, index=False로 저장해주세요.

In [ ]:
director_detail_df = pd.DataFrame(director_detail_list)
director_detail_df.to_csv('director_detail_2019_ko.csv', index=False)

In [ ]:
actor_detail_df = pd.DataFrame(actor_detail_list)
actor_detail_df.to_csv('actor_detail_2019_ko.csv', index=False)

In [ ]:
staff1_detail_df = pd.DataFrame(staff_detail_list_1)
staff1_detail_df.to_csv('staff1_detail_2019_ko.csv', index=False)

In [ ]:
# staff2_detail_df = pd.DataFrame(staff_detail_list_2)
# staff2_detail_df.to_csv('staff2_detail_2019_ko.csv', index=False)

In [ ]:
# staff3_detail_df = pd.DataFrame(staff_detail_list_3)
# staff3_detail_df.to_csv('staff3_detail_2019_ko.csv', index=False)

<br>\
<br>\
<br>

### 잘 돌아가다가 KeyError: 'peopleInfoResult'가 발생했을 때.

해당에러가 뜰 경우에는 key 사용 한도를 초과한 것입니다.

멈추기 전까지는 데이터가 다 저장되어있으니 당황하지 마시고 어디까지 저장되었는지 아래처럼 범위를 이용해서 또는 .tail() 함수를 이용해 데이터를 확인하고 다시 시작해야하는 부분이 어디인지 알아냅니다.

In [ ]:
# 데이터 확인
staff_detail_list_2[-10:]

In [ ]:
# '김효주', '아메리카 타운' 정보가 맨 마지막일 경우
# index값을 확인한다. 3000까지 뽑힌거니까 3001부터 데이터를 다시 가져오면 됨.
peopleCd_list[(peopleCd_list['peopleNm'] == '김효주') & (peopleCd_list['movieNm'] == '아메리카 타운')]